Importamos las librerias necesarias

In [1]:
import pandas as pd
import os

Obtenemos la ruta de la carpeta en la que estamos trabajando

Nos servira para abrir los archivos y luego guardarlos

In [2]:
# Construir la ruta completa al archivo
ruta_games = r'C:\PROYECTO-INDIVIDUAL\Data\json\sucio\output_steam_games.json'

# Abrimos el archivo y cargamos todas las lineas del mismo en "lineas"
with open(ruta_games, 'r', encoding='utf-8') as archivo:
    lineas = archivo.readlines()

# Leemos cada linea dentro de lineas, y realizamos la correcciones pertinentes
texto = []
for linea in lineas:
    texto_de_linea = linea.replace('"', '').replace('publisher: ',  '"publisher": "').replace(', genres: ',  '", "genres": "').replace(', app_name:',  '", "app_name": "').replace(', title:',  '", "title": "').replace(', url:',  '", "url": "').replace(', release_date:',  '", "release_date": "').replace(', tags:',  '", "tags": "').replace(', reviews_url:',  '", "reviews_url": "').replace(', specs:',  '", "specs": "').replace(', price:',  '", "price": "').replace(', early_access:',  '", "early_access": "').replace(', id:',  '", "id": "').replace(', developer:',  '", "developer": "').replace('NaN', 'None').replace('}', '"}').replace("\\", "").replace('"}{atch', 'atch').replace(";", ",")
    texto.append(texto_de_linea)

# Abrimos el archivo en formato escritura, cargamos todo el texto, y guardamos en una nueva ruta
ruta_nueva_games = r'C:\PROYECTO-INDIVIDUAL\Data\json\limpio\games.json'
archivo = open(ruta_nueva_games, "w", encoding='utf-8')
archivo.write('[' + '\n')
for i in range(len(texto)-1):
    archivo.write(texto[i] + ',')
archivo.write(texto[len(texto)-1])
archivo.write(']' + '\n')
archivo.close()

CREAMOS UN DATAFRAME CON ESE ARCHIVO

In [3]:
df_games = pd.read_json(ruta_nueva_games, encoding='utf-8')

REALIZAMOS UNA VISUALIZACION DE LOS DATOS, FILAS Y COLUMNAS

Con el fin de saber que columnas nos pueden servir de acuerdo a las actividades a realizar

In [4]:
print(df_games.shape)
print(df_games.columns)

(120445, 13)
Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')


AHORA ELIMINAREMOS COLUMNAS, VALORES NA, Y MODIFICAREMOS DATOS

In [5]:
# Eliminamos las columnas innecesarias
df_games.drop(columns=['url', 'reviews_url', 'specs', 'early_access', 'publisher', 'app_name'], inplace=True)

# Cambiar los valores de string 'None' por None
nuevos_valores = {'None': None, 'None ': None, ' None': None}
df_games['developer'] = df_games['developer'].replace(nuevos_valores)
# Eliminamos los valores NA
df_games.dropna(inplace=True)

# Ahora tenemos que separar los generos y crear variables dummy para mas facilidad a la hora de el manejo del archivo
# Función para convertir el string '[elemento1, elemento2, ..., elementon]' en lista
def convertir_a_lista(string_lista):
    # Eliminar corchetes y espacios
    string_lista = string_lista.replace("[", "").replace("]", "").replace(" ", "")
    # Dividir por coma y convertir a lista
    return string_lista.split(",")

# Aplicar la función a la columna 'genres'
df_games['genres'] = df_games['genres'].apply(convertir_a_lista)
# Separar la columna 'genres'
df_games.explode(column='genres')
# Convertir la lista de géneros en una cadena separada por comas
df_games['Generos_str'] = df_games['genres'].apply(lambda x: ', '.join(x))
df_games['genres'] = df_games['genres'].apply(lambda x: ', '.join(x))
# Obtener las variables dummy con el nombre de los géneros
df_dummies = df_games['Generos_str'].str.get_dummies(sep=', ')
# Unir las variables dummy con el DataFrame original
df_games = pd.concat([df_games, df_dummies], axis=1)
# Eliminar columnas temporales
df_games.drop(['Generos_str'], axis=1, inplace=True)

# Reemplazar NaN con 0 en la columna 'price'
df_games['price'] = df_games['price'].fillna(0)
# Reemplazar valores que contienen ciertos strings que complican el analisis con 0 en la columna 'price'
df_games.loc[df_games['price'].astype(str).str.contains('Free'), 'price'] = 0
df_games.loc[df_games['price'].astype(str).str.contains('Install'), 'price'] = 0
df_games.loc[df_games['price'].astype(str).str.contains('Demo'), 'price'] = 0
df_games.loc[df_games['price'].astype(str).str.contains('Third-party'), 'price'] = 0
df_games.loc[df_games['price'].astype(str).str.contains('Play'), 'price'] = 0
df_games.loc[df_games['price'].astype(str).str.contains('None'), 'price'] = 0
# Convertir la columna 'price' a tipo float
df_games['price'] = df_games['price'].astype(float)

# Convertir la columna 'release_date' a tipo date
df_games['fecha'] = pd.to_datetime(df_games['release_date'], errors='coerce')
# Filtrar las filas que tienen NaT en la columna 'fecha' (valores no válidos)
df_games = df_games.dropna(subset=['fecha'])
# Eliminar la columna original
df_games.drop(columns='release_date', inplace=True)
# Crear las nuevas columnas 'anio', 'mes' y 'dia' utilizando dt.year, dt.month y dt.day
df_games['anio'] = df_games['fecha'].dt.year
df_games['mes'] = df_games['fecha'].dt.month
df_games['dia'] = df_games['fecha'].dt.day
# Eliminar la columna fecha original
df_games.drop(columns='fecha', inplace=True)

# Transformar la columna 'developer' en minusculas y sin espacios en los bordes
df_games['developer'] = df_games['developer'].str.strip().str.lower()

# Eliminar el espacio en los bordes de la columna 'id'
df_games['id'] = df_games['id'].str.strip()
# Cambiar los 'None' por None en la columna id
df_games.loc[df_games['id'].astype(str).str.contains('None'), 'id'] = None
# Dropear los NA
df_games.dropna(subset=['id'], inplace=True)
# Transformar a entero la columna
df_games['id'] = df_games['id'].astype(int)

# Aplicar la función a la columna 'genres'
df_games['tags'] = df_games['tags'].apply(convertir_a_lista)
df_games['tags'] = df_games['tags'].apply(lambda x: ', '.join(x))

Vemos como quedo

In [6]:
df_games.head()

,genres,title,tags,price,id,developer,Accounting,Action,Adventure,Animation&amp,...,Simulation,SoftwareTraining,Sports,Strategy,Utilities,VideoProduction,WebPublishing,anio,mes,dia
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,"Strategy, Action, Indie, Casual, Simulation",4.99,761140,kotoshiro,0,1,0,0,...,1,0,0,1,0,0,0,2018,1,4
88311,"FreetoPlay, Indie, RPG, Strategy",Ironbound,"FreetoPlay, Strategy, Indie, RPG, CardGame, Tr...",0.00,643980,secret level srl,0,0,0,0,...,0,0,0,1,0,0,0,2018,1,4
88312,"Casual, FreetoPlay, Indie, Simulation, Sports",Real Pool 3D - Poolians,"FreetoPlay, Simulation, Sports, Casual, Indie,...",0.00,670290,poolians.com,0,0,0,0,...,1,0,1,0,0,0,0,2017,7,24
88313,"Action, Adventure, Casual",u5f39u70b8u4eba2222,"Action, Adventure, Casual",0.99,767400,u5f7cu5cb8u9886u57df,0,1,1,0,...,0,0,0,0,0,0,0,2017,12,7
88315,"Action, Adventure, Simulation",Battle Royale Trainer,"Action, Adventure, Simulation, FPS, Shooter, T...",3.99,772540,trickjump games ltd,0,1,1,0,...,1,0,0,0,0,0,0,2018,1,4


GUARDAMOS LOS DATAFRAMES

Para ello creamos una carpeta llamada parquet dentro de datos, y guardamos los archivos dentro de ella

In [7]:
ruta_de_guardado = r'C:\PROYECTO-INDIVIDUAL\Data\parquet\games.parquet'
df_games.to_parquet(ruta_de_guardado, index=False)